In [16]:
import sys, os
sys.path.append(os.path.abspath("../src"))

import pandas as pd
from optimization_model import nearest_neighbor_solution

In [17]:
orders_df = pd.read_csv("../data/sample/orders.csv")
vehicles_df = pd.read_csv("../data/sample/vehicles.csv")
depots_df = pd.read_csv("../data/sample/depots.csv")
distances = pd.read_csv("../data/sample/distances.csv", index_col=0)

In [18]:
dist_raw = pd.read_csv("../data/sample/distances.csv")        # or your distances file
dist_mat = pd.DataFrame()
if set(["from_id","to_id","distance_km"]).issubset(set(map(str.lower, dist_raw.columns))):
    # pivot will be handled inside the function, but we can call to_distance_matrix if needed.
    dist_mat = dist_raw  # pass long form — function will pivot
else:
    dist_mat = pd.read_csv("../data/sample/distances_square.csv", index_col=0)

orders = pd.read_csv("../data/sample/orders.csv")            # use sample for speed
vehicles = pd.read_csv("../data/sample/vehicles.csv")
depots = pd.read_csv("../data/sample/depots.csv")

In [19]:
orders["customer_id"] = orders["customer_id"].astype(str)
vehicles["vehicle_id"] = vehicles["vehicle_id"].astype(str)
depots["depot_id"] = depots["depot_id"].astype(str)

In [23]:
sol, tot_dist, tot_em, unserved = nearest_neighbor_solution(
    dist_mat,
    orders,
    vehicles,
    depots
)





⚠️ Skipping 88 customers not in distance matrix: ['C19', 'C74', 'C64', 'C30', 'C86']...


ValueError: not enough values to unpack (expected 4, got 3)

In [24]:
print("Total distance:", tot_dist)
print("Total emissions (kg):", tot_em)
print("Unserved count:", len(unserved))
print("Example vehicle route:", sol[0])

NameError: name 'tot_dist' is not defined

In [ ]:
NEW ONE

In [29]:
# 1) ensure src/ is importable
import sys, os
src_dir = os.path.abspath("../src")   # adjust if your notebook is not in notebooks/
print("Adding to sys.path:", src_dir)
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)

# 2) import and confirm which file Python sees
import importlib
import optimization_model
importlib.reload(optimization_model)   # force reload
print("optimization_model loaded from:", optimization_model.__file__)

# 3) inspect the function signature to confirm correct version
import inspect
print("signature:", inspect.signature(optimization_model.nearest_neighbor_solution_v2))


Adding to sys.path: /workspaces/FMCG---Route-Optimization-for-Sustainability/fmcg-route-optimization/src
optimization_model loaded from: /workspaces/FMCG---Route-Optimization-for-Sustainability/fmcg-route-optimization/src/optimization_model.py
signature: (distance_input: pandas.core.frame.DataFrame, orders_df: pandas.core.frame.DataFrame, vehicles_df: pandas.core.frame.DataFrame, depots_df: pandas.core.frame.DataFrame, demand_col: str = 'demand_units', vehicle_capacity_col: str = 'capacity_units', emission_col: str = 'co2_factor_kg_per_km', max_customers_per_vehicle: int | None = None) -> Tuple[List[Dict[str, Any]], float, float, List[str]]


In [37]:
from optimization_model import nearest_neighbor_solution_v2
import pandas as pd

# tiny sample distance matrix (must match ids used by orders/depot)
dist = pd.DataFrame([[0.0,5.0,10.0],[5.0,0.0,7.0],[10.0,7.0,0.0]],
                    index=["D0","C1","C2"], columns=["D0","C1","C2"])
orders = pd.DataFrame({"customer_id":["C1","C2"], "demand_units":[10,20]})
vehicles = pd.DataFrame({"vehicle_id":["V0"], "capacity_units":[100], "co2_factor_kg_per_km":[0.9]})
depots = pd.DataFrame({"depot_id":["D0"]})

sol, tot_dist, tot_em, unserved = nearest_neighbor_solution_v2(dist, orders, vehicles, depots)
print("sol:", sol)
print("tot_dist:", tot_dist)
print("tot_em:", tot_em)
print("unserved:", unserved)
print("--------------------------------------------------------------------------------------------")
print("=== Route Optimization Report ===")
print(f"Total Distance Traveled: {tot_dist:.2f} km")
print(f"Total Emissions: {tot_em:.2f} kg CO2")
print(f"Unserved Customers: {len(unserved)} -> {unserved if unserved else 'None'}")
print("\nVehicle Routes:")
for route in sol:
    path = " -> ".join(route["route"]) if route["route"] else "No customers assigned"
    print(
        f"  Vehicle {route['vehicle_id']} | "
        f"Load: {route['load']} units | "
        f"Customers served: {route['assigned_count']} | "
        f"Route: {path}"
    )

sol: [{'vehicle_id': 'V0', 'route': ['C1', 'C2', 'D0'], 'load': 30.0, 'assigned_count': 2}]
tot_dist: 22.0
tot_em: 19.8
unserved: []
--------------------------------------------------------------------------------------------
=== Route Optimization Report ===
Total Distance Traveled: 22.00 km
Total Emissions: 19.80 kg CO2
Unserved Customers: 0 -> None

Vehicle Routes:
  Vehicle V0 | Load: 30.0 units | Customers served: 2 | Route: C1 -> C2 -> D0


In [35]:
import pandas as pd

# Paths to your sample files
sample_dir = "../data/sample"  # adjust path if needed
distances_file = f"{sample_dir}/distances.csv"
emissions_file = f"{sample_dir}/emissions.csv"
orders_file = f"{sample_dir}/orders.csv"
vehicles_file = f"{sample_dir}/vehicles.csv"
depots_file = f"{sample_dir}/depots.csv"

# Load sample datasets
dist_mat = pd.read_csv(distances_file, index_col=0)  # Distance matrix with depot/customer IDs as index/columns
emissions_mat = pd.read_csv(emissions_file, index_col=0)
orders = pd.read_csv(orders_file)
vehicles = pd.read_csv(vehicles_file)
depots = pd.read_csv(depots_file)

# Optional: show a quick preview
print(dist_mat.head())
print(orders.head())
print(vehicles.head())
print(depots.head())


        to_id  distance_km    time_min
from_id                               
D0         D1     8.556738   17.113475
D0         C1    48.037610   96.075221
D0         C2    58.955078  117.910157
D0         C3    87.365981  174.731962
D0         C4    74.749098  149.498196
  order_id customer_id  demand_units temp_zone  priority  service_minutes  \
0       O0         C39           367   ambient         2               21   
1       O1          C8           275   chilled         1                8   
2       O2         C13           321   chilled         2                6   
3       O3         C33            99   ambient         2                7   
4       O4         C50           109   ambient         3                7   

   tw_start_min  tw_end_min  
0          1121        1255  
1           684         935  
2           435         674  
3           963        1033  
4          1087        1290  
  vehicle_id  capacity_units  reefer_supported  reefer_power_kw  fixed_cost  \
0    